In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import os
import zipfile
import shutil

In [2]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "download.default_directory": r"D:\Program Files (x86)\百度云同步盘\Dropbox\-E·J- 2014.5.1\2016.12.15 店小秘数据分析\2017.10.24 各渠道运费明细\燕文\\", # IMPORTANT - ENDING SLASH V IMPORTANT
         "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)

In [3]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [6]:
# 打开lagou
browser.get('https://www.lagou.com/wn/jobs?fromSearch=true&kd=%25E5%2588%2586%25E6%259E%2590&city=%E5%B9%BF%E5%B7%9E&pn=1')

In [7]:
def find_elements_by_css_selector(class_name, order_num):
    class_name = class_name.replace(" ", ".")
    class_name = '.' + class_name
    return browser.find_elements_by_css_selector(class_name)[order_num]

In [80]:
# 最大页数
max_page_num = int(browser.find_element_by_xpath("//div[@class='pagination__1L2PP']/ul[@class='lg-pagination']/li[last()-1]").text)
max_page_num

27

In [34]:
# 下一页
next_page_click = browser.find_element_by_xpath("//div[@class='pagination__1L2PP']/ul[@class='lg-pagination']/li[last()]")
next_page_click.click()

In [36]:
# 翻滚到最后一屏
body = browser.find_element_by_css_selector('body')
body.send_keys(Keys.END)
time.sleep(5)

In [71]:
# 职位标题
job_title = browser.find_elements_by_xpath("//div[@class='position__21iOS']/div[@class='p-top__1F7CL']/a")
job_title[0].text

'投资研究分析师[车陂]'

In [72]:
# 职位公司
job_company = browser.find_elements_by_xpath("//div[@class='company-name__2-SjF']")
job_company[0].text

'三七互娱'

In [73]:
# 职位价格
job_price = browser.find_elements_by_xpath("//span[@class='money__3Lkgq']")
job_price[0].text

'25k-40k'

In [ ]:
# 职位链接
# job_url = browser.find_element_by_xpath("//span[@class='money__3Lkgq']")
# job_url.text

In [128]:
# 广州
# browser.get('https://www.lagou.com/wn/jobs?fromSearch=true&kd=%25E5%2588%2586%25E6%259E%2590&city=%E5%B9%BF%E5%B7%9E&pn=1')
# 深圳
browser.get('https://www.lagou.com/wn/jobs?fromSearch=true&kd=%25E6%2595%25B0%25E6%258D%25AE&city=%E6%B7%B1%E5%9C%B3&pn=1')

job_short_desc = pd.DataFrame(columns={'job_location', 'job_title', 'job_company', 'job_price'})

for i in range(max_page_num):
    try:
        # 翻滚到最后一屏
        body = browser.find_element_by_css_selector('body')
        body.send_keys(Keys.END)
        time.sleep(10)
        # 职位标题
        job_title = browser.find_elements_by_xpath("//div[@class='position__21iOS']/div[@class='p-top__1F7CL']/a")
        # 职位公司
        job_company = browser.find_elements_by_xpath("//div[@class='company-name__2-SjF']")
        # 职位价格
        job_price = browser.find_elements_by_xpath("//span[@class='money__3Lkgq']")
        # 职位位置
        # job_location = '广州'
        job_location = '深圳'
        # 添加到dataframe
        for y in range(len(job_title)):
            job_short_desc = job_short_desc.append({'job_location': job_location,
                                                    'job_title': job_title[y].text,
                                                    'job_company': job_company[y].text,
                                                    'job_price': job_price[y].text
                                                   }, ignore_index=True)
        # 下一页
        next_page_click = browser.find_element_by_xpath("//div[@class='pagination__1L2PP']/ul[@class='lg-pagination']/li[last()]")
        next_page_click.click()    

        # print(job_short_desc.head(1))
    except:
        pass
    
job_short_desc.head()

,job_location,job_title,job_company,job_price
0,深圳,数据安全工程师[车公庙],OPPO,30k-45k
1,深圳,BI数据分析师[香蜜湖],平安产险深圳分公司,20k-25k
2,深圳,1141C0-大数据产品方案专家[南山区],平安科技,20k-35k
3,深圳,03516S-高级数据开发工程师-核心团队[福田区],平安产险,25k-35k
4,深圳,02514W-数据平台架构师[福田区],平安人寿,12k-24k


In [129]:
# 取最大值最小值
job_short_desc['job_price_min'] = job_short_desc['job_price'].str.split('-', expand=True)[0].str.split('k', expand=True)[0].astype(int)
job_short_desc['job_price_max'] = job_short_desc['job_price'].str.split('-', expand=True)[1].str.split('k', expand=True)[0].astype(int)

job_short_desc.head()

,job_location,job_title,job_company,job_price,job_price_min,job_price_max
0,深圳,数据安全工程师[车公庙],OPPO,30k-45k,30,45
1,深圳,BI数据分析师[香蜜湖],平安产险深圳分公司,20k-25k,20,25
2,深圳,1141C0-大数据产品方案专家[南山区],平安科技,20k-35k,20,35
3,深圳,03516S-高级数据开发工程师-核心团队[福田区],平安产险,25k-35k,25,35
4,深圳,02514W-数据平台架构师[福田区],平安人寿,12k-24k,12,24


In [130]:
# job_short_desc.to_csv('广州职位.csv', encoding='gb18030')
job_short_desc.to_csv('深圳职位.csv', encoding='gb18030')

In [131]:
job_short_desc['job_price_min'].describe()

count    405.000000
mean      19.595062
std       10.594179
min        3.000000
25%       15.000000
50%       20.000000
75%       20.000000
max      100.000000
Name: job_price_min, dtype: float64

In [132]:
job_short_desc['job_price_max'].describe()

count    405.000000
mean      33.333333
std       15.886202
min        4.000000
25%       25.000000
50%       30.000000
75%       40.000000
max      120.000000
Name: job_price_max, dtype: float64

In [133]:
job_short_desc.groupby(['job_company']).agg({
    'job_title': 'count',
    'job_price_min': 'mean',
    'job_price_max': 'mean'
}).reset_index().sort_values(by='job_price_min', ascending=False)

,job_company,job_price_min,job_title,job_price_max
169,金蝶信科,74.000000,3,90.000000
100,新大瀚人力资源,70.000000,3,100.000000
38,众安保险,55.000000,1,65.000000
157,跨越速运,46.666667,3,70.000000
22,vivo,40.000000,2,70.000000
44,兔展RABBITPRE,40.000000,5,70.000000
156,货拉拉,30.000000,2,47.500000
42,传智教育,30.000000,3,55.000000
46,前程无忧,30.000000,1,60.000000
0,360数科,30.000000,1,50.000000
